In [ ]:
!pip install -q kaggle av

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers@v4.49.0-SmolVLM-2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install num2words

In [ ]:
!pip install -q ninja packaging wheel
!pip install -q flash-attn --no-build-isolation

In [ ]:
!pip install -q gdown

In [ ]:
!pip install -q torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 83.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.


In [ ]:
from google.colab import drive
import os
import sys


try:
    drive.mount('/content/drive')
    print("Drive montado com sucesso.")
except Exception as e:
    print(f"ERRO: Falha ao montar o Drive: {e}")
    sys.exit()

DRIVE_FOLDER = '/content/drive/My Drive/D/Dataset/'
PASTA_VIDEOS_LOCAL = 'msrvtt_videos/TrainValVideo'
drive_zip_path = os.path.join(DRIVE_FOLDER, 'TrainValVideo.zip')
local_zip_path = 'TrainValVideo.zip'

if not os.path.exists(PASTA_VIDEOS_LOCAL):
    print(f"Pasta local '{PASTA_VIDEOS_LOCAL}' não encontrada. Iniciando descompactação...")
    if not os.path.exists(drive_zip_path):
        print(f"ERRO CRÍTICO: Arquivo ZIP '{drive_zip_path}' não encontrado no Google Drive.")
        sys.exit()
    else:
        print("Copiando arquivo ZIP do Drive para o ambiente Colab...")
        !cp '{drive_zip_path}' '{local_zip_path}'
        print("Descompactando arquivo ZIP...")
        !rm -rf 'msrvtt_videos'
        !unzip -q '{local_zip_path}' -d 'msrvtt_videos'
        print("dataset pronto")
else:
  print("pasta já existe, pulando a descompactação")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive montado com sucesso.
pasta já existe, pulando a descompactação


In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText
import torch
import os
from num2words import num2words


if not torch.cuda.is_available():
  raise SystemError("GPU não disponível. Mude o ambiente de execução para GPU.")

os.environ["HF_TOKEN"] = "coloque aqui sua chave token"
model_path = "HuggingFaceTB/SmolVLM2-2.2B-Instruct"

try:
    processor = AutoProcessor.from_pretrained(model_path)
    model = AutoModelForImageTextToText.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16).to("cuda")
    print("modelo carregado com sucesso")
except Exception as e:
    print(f"ERRO: Falha ao carregar o modelo: {e}")

    sys.exit()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

modelo carregado com sucesso


In [ ]:
import pandas as pd
from tqdm import tqdm
import csv

output_folder_drive = '/content/drive/My Drive/D/Dataset/'
output_filename = 'MSRVTT_descricoes_finais.csv'
full_drive_path = os.path.join(output_folder_drive, output_filename)

video_folder = 'msrvtt_videos/TrainValVideo'
todos_os_videos = sorted([f for f in os.listdir(video_folder) if f.endswith('.mp4')])
print(f"\n--- Iniciando verificação de progresso ---")
print(f"Encontrados {len(todos_os_videos)} vídeos no total.")

videos_ja_processados = set()
try:
    df_progresso = pd.read_csv(full_drive_path)
    videos_ja_processados = set(df_progresso['video_id'])
    print(f"Arquivo de progresso encontrado. {len(videos_ja_processados)} vídeos já foram processados.")
except FileNotFoundError:
    print(f"Arquivo '{output_filename}' não encontrado. Começando do zero.")
    # Cria o arquivo com o cabeçalho se ele não existir
    with open(full_drive_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['video_id', 'generated_description'])

videos_para_processar = [v for v in todos_os_videos if v not in videos_ja_processados]
print(f"Faltam {len(videos_para_processar)} vídeos para processar.")

if not videos_para_processar:
  print("Todos os vídeos já foram processados. Saindo.")

else:
  print(f"\n--- Iniciando processamento de {len(videos_para_processar)} vídeos restantes ---")

  with open(full_drive_path, mode='a', newline='', encoding='utf-8') as file:
      writer = csv.writer(file)

      for video_file in tqdm(videos_para_processar, desc="Processando Vídeos"):
          video_path = os.path.join(video_folder, video_file)

          messages = [{"role": "user", "content": [{"type": "video", "path": video_path}, {"type": "text", "text": "Describe this video in detail."}]},]

          try:
              inputs = processor.apply_chat_template(
                  messages, add_generation_prompt=True, tokenize=True,
                  return_dict=True, return_tensors="pt"
              ).to(model.device, dtype=torch.bfloat16)

              generated_ids = model.generate(**inputs, do_sample=False, max_new_tokens=100)
              generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
              description = generated_texts[0].strip()

              # Salva o resultado deste vídeo IMEDIATAMENTE
              writer.writerow([video_file, description])

          except Exception as e:
              print(f"\n  -> Falha ao processar o vídeo {video_file}: {e}")

              # Salva a informação de erro IMEDIATAMENTE
              writer.writerow([video_file, f'ERROR: {e}'])

  print("\n processamento dos vídeos restantes concluído")


--- Iniciando verificação de progresso ---
Encontrados 7010 vídeos no total.
Arquivo de progresso encontrado. 7010 vídeos já foram processados.
Faltam 0 vídeos para processar.
Todos os vídeos já foram processados. Saindo.


In [ ]:
import pandas as pd
import os
from google.colab import drive
import sys

print("--- Iniciando o script de limpeza do CSV ---")

# 1. Montar o Google Drive (se já não estiver montado)
try:
    if not os.path.exists('/content/drive/My Drive'):
        print("Montando Google Drive...")
        drive.mount('/content/drive')
    else:
        print("Google Drive já está montado.")
except Exception as e:
    print(f"ERRO: Falha ao montar o Drive: {e}")
    sys.exit() # Interrompe a execução se o Drive não puder ser montado

DRIVE_FOLDER = '/content/drive/My Drive/D/Dataset/'
CSV_FILENAME = 'MSRVTT_descricoes_finais.csv'
file_path = os.path.join(DRIVE_FOLDER, CSV_FILENAME)

try:
    print(f"\nLendo o arquivo: '{file_path}'")
    df = pd.read_csv(file_path)
    print(f"Arquivo lido com sucesso. Encontradas {len(df)} linhas.")

    # Exibir um exemplo da coluna ANTES da limpeza
    print("\nExemplo de descrição ANTES da limpeza:")

    primeira_descricao_antes = str(df['generated_description'].iloc[0])
    print(f"-> '{primeira_descricao_antes}'")

except FileNotFoundError:
    print(f"ERRO CRÍTICO: O arquivo '{CSV_FILENAME}' não foi encontrado no local esperado.")
    sys.exit()
except Exception as e:
    print(f"ERRO ao ler o arquivo CSV: {e}")
    sys.exit()


def limpar_descricao(texto):

    texto_str = str(texto)

    marca = "Assistant:"


    posicao = texto_str.find(marca)


    if posicao != -1:
        # Pega tudo o que vem DEPOIS da marca e seu comprimento
        texto_limpo = texto_str[posicao + len(marca):]
        # Remove espaços em branco extras do início e do fim
        return texto_limpo.strip()
    else:
        # Se a marca "Assistant:" não for encontrada, retorna o texto original sem alteração
        # Isso garante que descrições já limpas ou com erro não sejam apagadas
        return texto_str.strip()


print("\nAplicando a limpeza na coluna 'generated_description'...")
df['generated_description'] = df['generated_description'].apply(limpar_descricao)
print("Limpeza concluída.")

# Exibir um exemplo da coluna DEPOIS da limpeza para confirmação
print("\nExemplo de descrição DEPOIS da limpeza:")
primeira_descricao_depois = df['generated_description'].iloc[0]
print(f"-> '{primeira_descricao_depois}'")

# 6. Salvar o DataFrame limpo de volta no mesmo arquivo, sobrescrevendo-o
try:
    print(f"\nSalvando o arquivo atualizado em: '{file_path}'")
    # index=False evita que o Pandas adicione uma nova coluna de índice ao CSV
    df.to_csv(file_path, index=False, encoding='utf-8')
    print(" Arquivo atualizado e salvo com sucesso!")
except Exception as e:
    print(f"ERRO ao salvar o arquivo atualizado: {e}")

--- Iniciando o script de limpeza do CSV ---
Google Drive já está montado.

Lendo o arquivo: '/content/drive/My Drive/D/Dataset/MSRVTT_descricoes_finais.csv'
Arquivo lido com sucesso. Encontradas 7010 linhas.

Exemplo de descrição ANTES da limpeza:
-> 'The video begins with a person driving a car, wearing a dark shirt and a watch, with the interior of the car visible. The scene transitions to a nighttime setting with a road surrounded by trees, where a black car is seen driving away. The focus then shifts to a car's dashboard screen displaying options for "Anheben" (start) and "Borduch" (stop), with "Audi drive select" highlighted. The screen also shows "Car Systeme" (car systems'

Aplicando a limpeza na coluna 'generated_description'...
Limpeza concluída.

Exemplo de descrição DEPOIS da limpeza:
-> 'The video begins with a person driving a car, wearing a dark shirt and a watch, with the interior of the car visible. The scene transitions to a nighttime setting with a road surrounded by

- gerar os keyframes dos videos trainvalvideo.zip no drive sem gpu pois não precisa

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from google.colab import drive
import zipfile

print("Iniciando Script de Extração de Keyframes")

if not os.path.exists('/content/drive/My Drive'):
    print("Montando Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive já está montado.")

ZIP_FILE_PATH_ON_DRIVE = '/content/drive/MyDrive/D/Dataset/TrainValVideo.zip'
LOCAL_VIDEO_FOLDER = '/content/MSRVTT_Videos_descompactados'

print("\nEtapa de Descompactação")
print(f"Procurando pelo arquivo ZIP em: {ZIP_FILE_PATH_ON_DRIVE}")

if os.path.exists(LOCAL_VIDEO_FOLDER) and os.listdir(LOCAL_VIDEO_FOLDER):
    print(f"A pasta '{LOCAL_VIDEO_FOLDER}' já existe. Pulando descompactação.")
else:
    if not os.path.exists(ZIP_FILE_PATH_ON_DRIVE):
        print(f"ERRO: O arquivo ZIP não foi encontrado no caminho especificado.")
        exit()

    print("Arquivo ZIP encontrado. Iniciando descompactação para o ambiente local...")
    os.makedirs(LOCAL_VIDEO_FOLDER, exist_ok=True)

    with zipfile.ZipFile(ZIP_FILE_PATH_ON_DRIVE, 'r') as zip_ref:
        zip_ref.extractall(LOCAL_VIDEO_FOLDER)
    print("Descompactação concluída.")

KEYFRAME_DESTINATION_FOLDER = '/content/drive/My Drive/D/Dataset/Keyframes'
NUM_KEYFRAMES_TO_EXTRACT = 8

os.makedirs(KEYFRAME_DESTINATION_FOLDER, exist_ok=True)
print(f"\nKeyframes serão salvos em: {KEYFRAME_DESTINATION_FOLDER}")

VIDEO_SOURCE_FOLDER = LOCAL_VIDEO_FOLDER

try:
    potential_subfolder = os.path.join(VIDEO_SOURCE_FOLDER, 'TrainValVideo')
    if os.path.exists(potential_subfolder) and os.path.isdir(potential_subfolder):
        print(f"Subpasta 'TrainValVideo' encontrada. Usando como fonte de vídeos.")
        VIDEO_SOURCE_FOLDER = potential_subfolder

    todos_os_videos = sorted([f for f in os.listdir(VIDEO_SOURCE_FOLDER) if f.endswith('.mp4')])
    if not todos_os_videos:
        raise FileNotFoundError

    print("\nVerificação de Progresso")
    print(f"Encontrados {len(todos_os_videos)} vídeos no total em '{VIDEO_SOURCE_FOLDER}'.")

except FileNotFoundError:
    print(f"ERRO: Nenhum vídeo .mp4 encontrado na pasta descompactada '{VIDEO_SOURCE_FOLDER}'.")
    exit()

videos_ja_processados = set(os.listdir(KEYFRAME_DESTINATION_FOLDER))
videos_para_processar = []

for video_file in todos_os_videos:
    video_id = os.path.splitext(video_file)[0]
    if video_id not in videos_ja_processados:
        videos_para_processar.append(video_file)

print(f"{len(videos_ja_processados)} vídeos já foram processados.")
print(f"Faltam {len(videos_para_processar)} vídeos para processar.")

if not videos_para_processar:
    print("\nTodos os vídeos já foram processados. Nenhuma ação necessária.")
else:
    print(f"\nIniciando extração de keyframes para {len(videos_para_processar)} vídeos restantes.")
    erros_de_processamento = []

    for video_file in tqdm(videos_para_processar, desc="Extraindo Keyframes"):
        video_id = os.path.splitext(video_file)[0]
        video_path = os.path.join(VIDEO_SOURCE_FOLDER, video_file)
        output_subfolder = os.path.join(KEYFRAME_DESTINATION_FOLDER, video_id)

        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                raise IOError("Não foi possível abrir o arquivo de vídeo.")

            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            os.makedirs(output_subfolder, exist_ok=True)
            frame_indices = np.linspace(0, total_frames - 1, NUM_KEYFRAMES_TO_EXTRACT, dtype=int)

            for i, frame_index in enumerate(frame_indices):
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
                ret, frame = cap.read()
                if ret:
                    keyframe_filename = f"frame_{i+1:02d}.jpg"
                    keyframe_save_path = os.path.join(output_subfolder, keyframe_filename)
                    cv2.imwrite(keyframe_save_path, frame)

            cap.release()

        except Exception as e:
            print(f"\nFalha ao processar o vídeo {video_file}: {e}")
            erros_de_processamento.append(f"{video_file}: {e}")
            if os.path.exists(output_subfolder):
                try:
                    os.rmdir(output_subfolder)
                except OSError:
                    pass

    print("\nProcessamento concluído.")
    if erros_de_processamento:
        print("\nOcorreram os seguintes erros durante a extração:")
        for erro in erros_de_processamento:
            print(f"  - {erro}")
    else:
        print("Todos os vídeos restantes foram processados com sucesso.")

Iniciando Script de Extração de Keyframes
Google Drive já está montado.

Etapa de Descompactação
Procurando pelo arquivo ZIP em: /content/drive/MyDrive/D/Dataset/TrainValVideo.zip
Arquivo ZIP encontrado. Iniciando descompactação para o ambiente local...
Descompactação concluída.

Keyframes serão salvos em: /content/drive/My Drive/D/Dataset/Keyframes
Subpasta 'TrainValVideo' encontrada. Usando como fonte de vídeos.

Verificação de Progresso
Encontrados 7010 vídeos no total em '/content/MSRVTT_Videos_descompactados/TrainValVideo'.
7010 vídeos já foram processados.
Faltam 0 vídeos para processar.

Todos os vídeos já foram processados. Nenhuma ação necessária.


- CLIP gerando embeddings agora com GPU t4

In [ ]:
import os
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm
from google.colab import drive

print("Iniciando o script de extração de features com CLIP...")

try:
    drive.mount('/content/drive')
    print("Google Drive montado com sucesso.")
except Exception as e:
    print(f"Erro ao montar o Google Drive: {e}")
    raise

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU detectada: {gpu_name}")
else:
    print("AVISO: Nenhuma GPU detectada. O processo será extremamente lento.")

MODEL_NAME = 'openai/clip-vit-large-patch14'

print(f"\nCarregando o modelo CLIP '{MODEL_NAME}'...")
print("Isso pode levar alguns minutos, especialmente na primeira execução...")

try:
    model = CLIPModel.from_pretrained(MODEL_NAME).to(device)
    processor = CLIPProcessor.from_pretrained(MODEL_NAME)
    model.eval()
    print("Modelo e processador carregados e movidos para o dispositivo com sucesso.")
except Exception as e:
    print(f"\n!!! ERRO CRÍTICO DURANTE O CARREGAMENTO DO MODELO !!!")
    print(f"Erro: {e}")
    raise

BASE_PATH = '/content/drive/MyDrive/D/Dataset'
KEYFRAMES_FOLDER = os.path.join(BASE_PATH, 'Keyframes')

FEATURES_SEQUENTIAL_FOLDER = os.path.join(BASE_PATH, 'MSRVTT_Features_ViT_L14_Sequential')
FEATURES_AGGREGATED_FOLDER = os.path.join(BASE_PATH, 'MSRVTT_Features_ViT_L14_Aggregated')

os.makedirs(FEATURES_SEQUENTIAL_FOLDER, exist_ok=True)
os.makedirs(FEATURES_AGGREGATED_FOLDER, exist_ok=True)

print(f"\nPastas de entrada e saída configuradas.")
print(f"Keyframes lidos de: {KEYFRAMES_FOLDER}")
print(f"Features Sequenciais salvas em: {FEATURES_SEQUENTIAL_FOLDER}")
print(f"Features Agregadas salvas em: {FEATURES_AGGREGATED_FOLDER}")

print("\nIniciando a extração de features. Isso pode levar um tempo...")

video_ids = sorted(os.listdir(KEYFRAMES_FOLDER))

with torch.no_grad():
    for video_id in tqdm(video_ids, desc="Processando Vídeos"):
        video_keyframes_path = os.path.join(KEYFRAMES_FOLDER, video_id)

        if not os.path.isdir(video_keyframes_path):
            continue

        agg_output_path = os.path.join(FEATURES_AGGREGATED_FOLDER, f"{video_id}.npy")
        seq_output_folder = os.path.join(FEATURES_SEQUENTIAL_FOLDER, video_id)

        agg_exists = os.path.exists(agg_output_path)
        seq_exists = os.path.exists(seq_output_folder)

        if agg_exists and seq_exists:
            continue

        frame_files = sorted([f for f in os.listdir(video_keyframes_path) if f.endswith(('.png', '.jpg', '.jpeg'))])

        if len(frame_files) != 8:
            continue

        images = [Image.open(os.path.join(video_keyframes_path, f)) for f in frame_files]

        inputs = processor(text=None, images=images, return_tensors="pt", padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        image_features = model.get_image_features(**inputs)
        image_features_np = image_features.cpu().numpy()

        if not seq_exists:
            os.makedirs(seq_output_folder, exist_ok=True)
            for i, feature_vector in enumerate(image_features_np):
                frame_num = i + 1
                feature_filename = f"feature_{frame_num:02d}.npy"
                feature_save_path = os.path.join(seq_output_folder, feature_filename)
                np.save(feature_save_path, feature_vector)

        if not agg_exists:
            mean_feature = np.mean(image_features_np, axis=0)
            np.save(agg_output_path, mean_feature)

print("\n\n===================================")
print("Extração de features concluída com sucesso!")
print("===================================")

Iniciando o script de extração de features com CLIP...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado com sucesso.
GPU detectada: Tesla T4

Carregando o modelo CLIP 'openai/clip-vit-large-patch14'...
Isso pode levar alguns minutos, especialmente na primeira execução...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Modelo e processador carregados e movidos para o dispositivo com sucesso.

Pastas de entrada e saída configuradas.
Keyframes lidos de: /content/drive/MyDrive/D/Dataset/Keyframes
Features Sequenciais salvas em: /content/drive/MyDrive/D/Dataset/MSRVTT_Features_ViT_L14_Sequential
Features Agregadas salvas em: /content/drive/MyDrive/D/Dataset/MSRVTT_Features_ViT_L14_Aggregated

Iniciando a extração de features. Isso pode levar um tempo...


Processando Vídeos:   0%|          | 0/7010 [00:00<?, ?it/s]



Extração de features concluída com sucesso!


- Dataframe unificado:
1. Fonte 1 (LLM): Carrega as descrições geradas pela LLM de MSRVTT_descricoes_finais.csv.
2. Fonte 2 (Original): Carrega as anotações do train_val_videodatainfo.json.
3. Fonte 3 (Visual): Gera caminhos relativos para todas as features visuais, garantindo portabilidade. Uma coluna para a feature agregada (agg_feature_path).
Oito colunas separadas para as features sequenciais (seq_feature_path_1 a seq_feature_path_8).




In [ ]:
import pandas as pd
import json
import os
import sys
from google.colab import drive

# --- Bloco 1: Montagem do Google Drive ---
if not os.path.exists('/content/drive/My Drive'):
    print("Montando Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive já está montado.")

# --- Bloco 2: Definição de Caminhos ---
DRIVE_FOLDER = '/content/drive/My Drive/D/Dataset/'
LLM_CSV_PATH = os.path.join(DRIVE_FOLDER, 'MSRVTT_descricoes_finais.csv')
MSRVTT_JSON_PATH = os.path.join(DRIVE_FOLDER, 'train_val_videodatainfo.json')

# --- Bloco 3: Carregamento e Processamento das Descrições da LLM ---
try:
    print(f"\nCarregando descrições geradas pela LLM de: {LLM_CSV_PATH}")
    df_llm = pd.read_csv(LLM_CSV_PATH)
    df_llm = df_llm.rename(columns={'generated_description': 'desc_llm'})
    if 'id' in df_llm.columns and 'video_id' not in df_llm.columns:
        df_llm = df_llm.rename(columns={'id': 'video_id'})

    df_llm['video_id'] = df_llm['video_id'].str.replace('.mp4', '', regex=False)
    print(f"Encontradas {len(df_llm)} descrições geradas.")

except FileNotFoundError:
    print(f"Arquivo não encontrado: {LLM_CSV_PATH}")
    sys.exit()

# --- Bloco 4: Carregamento e Processamento das Anotações Humanas ---
try:
    print(f"\nCarregando anotações humanas de: {MSRVTT_JSON_PATH}")
    with open(MSRVTT_JSON_PATH, 'r') as f:
        data = json.load(f)
    annotations = [{'video_id': item['video_id'], 'desc_human': item['caption']} for item in data['sentences']]
    df_human_raw = pd.DataFrame(annotations)
    print(f"   - Total de {len(df_human_raw)} anotações humanas carregadas.")

    df_human_raw['desc_num'] = df_human_raw.groupby('video_id').cumcount() + 1
    df_human_wide = df_human_raw.pivot(index='video_id', columns='desc_num', values='desc_human')
    df_human_wide.columns = [f'desc_human_{col}' for col in df_human_wide.columns]
    df_human_wide = df_human_wide.reset_index()
    print(f"   - Formato 'largo' criado com {len(df_human_wide.columns) - 1} colunas de descrição.")

    video_info = [{'video_id': video['video_id'], 'category': video['category']} for video in data['videos']]
    df_categories = pd.DataFrame(video_info)
    print(f"   - {len(df_categories)} categorias de vídeo extraídas.")

except FileNotFoundError:
    print(f"Arquivo JSON não encontrado: {MSRVTT_JSON_PATH}")
    sys.exit()

# --- Bloco 5: Unificação dos Dados Textuais ---
print("\nJuntando dados textuais em um DataFrame mestre...")
df_master = pd.merge(df_llm, df_categories, on='video_id', how='left')
df_master = pd.merge(df_master, df_human_wide, on='video_id', how='left')

# --- Bloco 6: Adição dos CAMINHOS RELATIVOS das features ---
print("Adicionando caminhos RELATIVOS das features visuais ao DataFrame...")

df_master['agg_feature_path'] = df_master['video_id'].apply(
    lambda x: os.path.join('MSRVTT_Features_ViT_L14_Aggregated', f"{x}.npy")
)

NUM_SEQUENTIAL_FEATURES = 8
for i in range(NUM_SEQUENTIAL_FEATURES):
    col_name = f'seq_feature_path_{i+1}'
    df_master[col_name] = df_master['video_id'].apply(
        lambda video_id: os.path.join('MSRVTT_Features_ViT_L14_Sequential', video_id, f'feature_{i}.npy')
    )
print(f"   - {NUM_SEQUENTIAL_FEATURES} colunas para features sequenciais adicionadas com sucesso.")


# --- Bloco 7: Organização Final e Salvamento ---
print("Reorganizando colunas para uma ordem lógica...")

# Define a ordem desejada das colunas de forma explícita
seq_feature_cols = [f'seq_feature_path_{i+1}' for i in range(NUM_SEQUENTIAL_FEATURES)]
human_desc_cols = sorted([col for col in df_master.columns if col.startswith('desc_human_')])

# Constrói a lista final na ordem solicitada
cols_ordered = [
    'video_id',
    'category',
    'agg_feature_path',  # Feature agregada primeiro
] + seq_feature_cols + [ # Depois as features sequenciais
    'desc_llm',          # Descrição da LLM primeiro
] + human_desc_cols      # Depois as descrições humanas

# Garante que estamos usando apenas colunas que realmente existem no DataFrame
cols_ordered = [col for col in cols_ordered if col in df_master.columns]

df_master = df_master[cols_ordered]

MASTER_CSV_PATH = os.path.join(DRIVE_FOLDER, 'MSRVTT_dados_compilados_com_features.csv')
df_master.to_csv(MASTER_CSV_PATH, index=False, encoding='utf-8')
print(f"\nSUCESSO! DataFrame com nova ordem de colunas salvo em: {MASTER_CSV_PATH}")

# --- Bloco 8: Visualização ---
print("\n--- Visualização do DataFrame Final (primeiras 5 linhas) ---")
pd.set_option('display.max_columns', 40)
display(df_master.head())

Google Drive já está montado.

Carregando descrições geradas pela LLM de: /content/drive/My Drive/D/Dataset/MSRVTT_descricoes_finais.csv
Encontradas 7010 descrições geradas.

Carregando anotações humanas de: /content/drive/My Drive/D/Dataset/train_val_videodatainfo.json
   - Total de 140200 anotações humanas carregadas.
   - Formato 'largo' criado com 20 colunas de descrição.
   - 7010 categorias de vídeo extraídas.

Juntando dados textuais em um DataFrame mestre...
Adicionando caminhos RELATIVOS das features visuais ao DataFrame...
   - 8 colunas para features sequenciais adicionadas com sucesso.
Reorganizando colunas para uma ordem lógica...

SUCESSO! DataFrame com nova ordem de colunas salvo em: /content/drive/My Drive/D/Dataset/MSRVTT_dados_compilados_com_features.csv

--- Visualização do DataFrame Final (primeiras 5 linhas) ---


,video_id,category,agg_feature_path,seq_feature_path_1,seq_feature_path_2,seq_feature_path_3,seq_feature_path_4,seq_feature_path_5,seq_feature_path_6,seq_feature_path_7,seq_feature_path_8,desc_llm,desc_human_1,desc_human_10,desc_human_11,desc_human_12,desc_human_13,desc_human_14,desc_human_15,desc_human_16,desc_human_17,desc_human_18,desc_human_19,desc_human_2,desc_human_20,desc_human_3,desc_human_4,desc_human_5,desc_human_6,desc_human_7,desc_human_8,desc_human_9
0,video0,9,MSRVTT_Features_ViT_L14_Aggregated/video0.npy,MSRVTT_Features_ViT_L14_Sequential/video0/feat...,MSRVTT_Features_ViT_L14_Sequential/video0/feat...,MSRVTT_Features_ViT_L14_Sequential/video0/feat...,MSRVTT_Features_ViT_L14_Sequential/video0/feat...,MSRVTT_Features_ViT_L14_Sequential/video0/feat...,MSRVTT_Features_ViT_L14_Sequential/video0/feat...,MSRVTT_Features_ViT_L14_Sequential/video0/feat...,MSRVTT_Features_ViT_L14_Sequential/video0/feat...,"The video begins with a person driving a car, ...",a car is shown,a man riding the car speedly in a narrow road,a man showing the various features of a car,a man silently narrates his experience driving...,a person is driving his car around curves in t...,a person telling about a car,guy driving a car down the road,man talking about a car while driving,the man drives the car,the man driving the audi as smooth as possible,a man is driving,a group is dancing,guy driving a car down the road,a man drives a vehicle through the countryside,a man drives down the road in an audi,a man driving a car,a man is driving a car,a man is driving down a road,a man is driving in a car as part of a commercial,a man is driving
1,video1,16,MSRVTT_Features_ViT_L14_Aggregated/video1.npy,MSRVTT_Features_ViT_L14_Sequential/video1/feat...,MSRVTT_Features_ViT_L14_Sequential/video1/feat...,MSRVTT_Features_ViT_L14_Sequential/video1/feat...,MSRVTT_Features_ViT_L14_Sequential/video1/feat...,MSRVTT_Features_ViT_L14_Sequential/video1/feat...,MSRVTT_Features_ViT_L14_Sequential/video1/feat...,MSRVTT_Features_ViT_L14_Sequential/video1/feat...,MSRVTT_Features_ViT_L14_Sequential/video1/feat...,The video begins with a close-up of a pot fill...,in a kitchen a woman adds different ingredient...,a woman cooking food with a metal pan on top o...,a woman adds different ingredients into a a po...,a woman in a kitchen is cooking a stew in a la...,a women in a multi-color outfit is cooking a s...,a woman adds ingredients to a pot that is simm...,a woman is preparing a seafood stew recipe on ...,in a kitchen a lady preferred crayfish with mi...,a woman and a bowl spoon mixing dish inside ...,cooking the dried smoked prawn in a vessel hav...,a lady is making dried prawns curry and she ad...,a woman puts prawns and seasonings into a larg...,a woman in a colorful scarf is showing how to ...,in the kitchen a woman makes a dish by adding ...,a woman adding ingredients to a pot on the sto...,instructions on how to cook a dish of prawns o...,a woman is in the kitchen making a recipe in a...,a woman adds some packets of spices and spoonf...,a person add ingredients to a pot in a counter...,a person puts items in a pot on the stove in t...
2,video10,6,MSRVTT_Features_ViT_L14_Aggregated/video10.npy,MSRVTT_Features_ViT_L14_Sequential/video10/fea...,MSRVTT_Features_ViT_L14_Sequential/video10/fea...,MSRVTT_Features_ViT_L14_Sequential/video10/fea...,MSRVTT_Features_ViT_L14_Sequential/video10/fea...,MSRVTT_Features_ViT_L14_Sequential/video10/fea...,MSRVTT_Features_ViT_L14_Sequential/video10/fea...,MSRVTT_Features_ViT_L14_Sequential/video10/fea...,MSRVTT_Features_ViT_L14_Sequential/video10/fea...,The video starts with a group of people standi...,a man holds two dogs,a man squatting with two dogs tells someone ha...,a man talking to the camera with two dogs next...,a man talks and plays with his two dogs,a man with wo dogs is speaking to the camera,a person is with his pets,a person wishes someone a happy birthday while...,a woman sings while on the floor with her lapt...,man introd